In [1]:
!which pip
!pwd

/usr/local/bin/pip
/content


In [199]:
import os
import atexit
import sys
import time
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
import findspark
from sparkhpc import sparkjob

#Exit handler to clean up the Spark cluster if the script exits or crashes
def exitHandler(sj,sc):
    try:
        print('Trapped Exit cleaning up Spark Context')
        sc.stop()
    except:
        pass
    try:
        print('Trapped Exit cleaning up Spark Job')
        sj.stop()
    except:
        pass

findspark.init()

#Parameters for the Spark cluster
nodes=1
tasks_per_node=4
memory_per_task=4096 #4 gig per process, adjust accordingly
# Please estimate walltime carefully to keep unused Spark clusters from sitting 
# idle so that others may use the resources.
walltime="15:00" #12 hours
#os.environ['SBATCH_PARTITION']='cpu2019' #Set the appropriate ARC partition

sj = sparkjob.sparkjob(
     ncores=nodes*tasks_per_node,
     cores_per_executor=tasks_per_node,
     memory_per_core=memory_per_task,
     walltime=walltime
    )

sj.wait_to_start()
time.sleep(60)

sc = sj.start_spark()

#Register the exit handler                                                                                                     
atexit.register(exitHandler,sj,sc)

#You need this line if you want to use SparkSQL
sqlCtx=SQLContext(sc)


INFO:sparkhpc.sparkjob:Submitted batch job 12529

INFO:sparkhpc.sparkjob:Submitted cluster 6
INFO:sparkhpc.sparkjob:


SystemExit: ignored

/global/software/jupyterhub-spark/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [103]:
sqlContext = sqlCtx
spark = sqlCtx

NameError: ignored

In [124]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [125]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

In [126]:
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [127]:
!pip install -q findspark

In [128]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [129]:
import findspark
findspark.init()

In [134]:
spark

ConnectionRefusedError: ignored

In [135]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab1")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

ConnectionRefusedError: ignored

In [22]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [107]:
spark

ConnectionRefusedError: ignored

In [106]:
#!pip install --upgrade pip
!pip install nltk
import nltk
nltk.download('punkt')
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [108]:
def tokenize1(text):
    words = nltk.word_tokenize(text)
    return words  
tokenize_word = udf(lambda x: tokenize1(x)  , ArrayType(StringType()))

In [109]:
def tokenize2(text):
    sents = nltk.sent_tokenize(text)
    return sents  
tokenize_sent = udf(lambda x: tokenize2(x)  , ArrayType(StringType()))

In [110]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_en = stopwords.words('english')
def remove_stopwords1(word_list):
    filtered_words = [word for word in word_list if word not in stop_en]
    return filtered_words
remove_stopwords = udf(lambda x: remove_stopwords1(x) , ArrayType(StringType()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
def remove_noise1(word_list):
    filtered_words = [word for word in word_list if word.isalnum() and len(word)>2]
    return filtered_words
remove_noise = udf(lambda x: remove_noise1(x) , ArrayType(StringType()))

In [112]:
from nltk.stem import SnowballStemmer
def stem1(word_list):
    snowball = SnowballStemmer(language='english')
    stemmed_words = [snowball.stem(word) for word in word_list]
    return stemmed_words
stem = udf(lambda x: stem1(x) , ArrayType(StringType()))

In [113]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
def sentiment1(text):
  sia = SentimentIntensityAnalyzer()
  return sia.polarity_scores(text)['compound']
sentiment = udf(lambda x: sentiment1(x) , FloatType())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [114]:
#area for testing functions:

mylist = ['GeeksforGeeks', ",", 'is', 'a', 'portal', 'for', 'geeks']
alphanum_words = [word for word in mylist if word.isalnum()]
N = len(alphanum_words)
punctuation_words = [word for word in mylist if not(word.isalnum())]
num_punctuation_words = len(punctuation_words)
ratio_punctuation_to_total = num_punctuation_words / N
ratio_punctuation_to_total

0.16666666666666666

In [115]:
#greg
#create udf for adding length of reviews by word count for all alphanumeric "words"

def get_length1(word_list):
  alphanum_words = [word for word in word_list if word.isalnum()]
  N = len(alphanum_words)
  return N

get_length = udf(lambda x: get_length1(x), IntegerType())

In [116]:
#greg
#create udf for adding average word length by review

def get_average_word_length1(word_list):
  alphanum_word_lengths = [len(word) for word in word_list if word.isalnum()]
  avg_word_len = sum(alphanum_word_lengths)/len(alphanum_word_lengths)
  return avg_word_len

get_average_word_length = udf(lambda x: get_average_word_length1(x), FloatType())

In [117]:
#greg
#create udf for ratio of capitalized characters to total characters (n), with no white spaces

def capital_ratio1(word_list):
  s = "".join(word_list)
  n = len(s)
  cap_n = len([char for char in s if char.isupper()])
  ratio_cap_to_total = cap_n / n
  return ratio_cap_to_total

capital_ratio = udf(lambda x: capital_ratio1(x), FloatType())


In [118]:
#greg
#create udf for ratio of long words (>5 letters) to total words (N)

def long_word_ratio1(word_list):
  alphanum_words = [word for word in word_list if word.isalnum()]
  N = len(alphanum_words)
  long_words = [word for word in alphanum_words if len(word) > 5]
  long_words_count = len(long_words)
  ratio_long_to_total = long_words_count / N
  return ratio_long_to_total

long_word_ratio = udf(lambda x: long_word_ratio1(x), FloatType())

In [119]:
#greg
#create udf for ratio of words after stop word removal vs total words

stop_en = stopwords.words('english')

def filtered_word_ratio1(word_list):
  alphanum_words = [word for word in word_list if word.isalnum()]
  N = len(alphanum_words)
  non_filtered_words = [word for word in alphanum_words if word not in stop_en]
  num_non_filtered_words = len(non_filtered_words)
  ratio_non_filtered_to_total = num_non_filtered_words / N
  return ratio_non_filtered_to_total

filtered_word_ratio = udf(lambda x: filtered_word_ratio1(x), FloatType())


In [120]:
#greg
#create udf for ratio of punctuation "words" vs total alphanumeric words

def punctuation_word_ratio1(word_list):
  alphanum_words = [word for word in word_list if word.isalnum()]
  N = len(alphanum_words)
  punctuation_words = [word for word in word_list if not(word.isalnum())]
  num_punctuation_words = len(punctuation_words)
  ratio_punctuation_to_total = num_punctuation_words / N
  return ratio_punctuation_to_total

punctuation_word_ratio = udf(lambda x: punctuation_word_ratio1(x), FloatType())

In [121]:
from pyspark.sql.functions import lower
def process_data(df):
  dfText = df.select("Index","Review" ,"polarity","real_fake", tokenize_word("Review").alias("tokenized_words"))
  dfText = dfText.withColumn("sentiment" ,sentiment("Review"))
  dfText = dfText.withColumn("tokenized_sents" ,tokenize_sent("Review"))
  dfText = dfText.withColumn("no_stopwords", remove_stopwords("tokenized_words"))
  dfText = dfText.withColumn("no_noise", remove_noise("no_stopwords"))
  dfText = dfText.withColumn("stemmed", stem("no_noise"))
  #dfText = dfText.select('*', F.concat_ws("_","real_fake","polarity").alias("target"))
  dfText = dfText.withColumn("length_in_words", get_length("tokenized_words"))
  dfText = dfText.withColumn("average_word_length", get_average_word_length("tokenized_words"))
  dfText = dfText.withColumn("capital_char_ratio", capital_ratio("tokenized_words"))
  dfText = dfText.withColumn("long_word_ratio", long_word_ratio("tokenized_words"))
  dfText = dfText.withColumn("non_stop_word_ratio", filtered_word_ratio("tokenized_words"))
  dfText = dfText.withColumn("punctuation_ratio", punctuation_word_ratio("tokenized_words"))

  return dfText

In [41]:
df_raw_old = spark.read.option("escape","\"").option("header",True).csv("Original_data.csv")
df_old_all = process_data(df_raw_old)
#df_test = df_test_all.select("Review","stemmed","sentiment","target","real_fake")
df_old = df_old_all.select("Review","stemmed","sentiment","polarity", "length_in_words", "average_word_length", \
                             "capital_char_ratio", "long_word_ratio", "non_stop_word_ratio", "punctuation_ratio", "real_fake")

old_train_split, old_test_split = df_old.randomSplit(weights = [0.80, 0.20], seed = 1)
old_train_split.cache()
old_test_split.cache()

df_raw_new = spark.read.option("escape","\"").option("header",True).csv("Hotel_Reviews_Calgary.csv")
df_new_all = process_data(df_raw_new)
#df_train = df_train.select("Review","stemmed","sentiment","target","real_fake")
df_new = df_new_all.select("Review","stemmed","sentiment","polarity", "length_in_words", "average_word_length", \
                             "capital_char_ratio", "long_word_ratio", "non_stop_word_ratio", "punctuation_ratio", "real_fake")

new_train_split, new_test_split = df_new.randomSplit(weights = [0.80, 0.20], seed = 1)
new_train_split.cache()
new_test_split.cache()

#df_train.display()
combined_train = old_train_split.union(new_train_split)
combined_test = old_test_split.union(new_test_split)

combined_train.cache()
combined_test.cache()

DataFrame[Review: string, stemmed: array<string>, sentiment: float, polarity: string, length_in_words: int, average_word_length: float, capital_char_ratio: float, long_word_ratio: float, non_stop_word_ratio: float, punctuation_ratio: float, real_fake: string]

In [42]:
print((combined_train.count(), len(combined_train.columns)))

(2119, 11)


In [43]:
combined_train.show(1)

+--------------------+--------------------+---------+--------+---------------+-------------------+------------------+---------------+-------------------+-----------------+---------+
|              Review|             stemmed|sentiment|polarity|length_in_words|average_word_length|capital_char_ratio|long_word_ratio|non_stop_word_ratio|punctuation_ratio|real_fake|
+--------------------+--------------------+---------+--------+---------------+-------------------+------------------+---------------+-------------------+-----------------+---------+
| Barely Average H...|[bare, averag, ho...|   0.1901|negative|            187|           4.390374|         0.0391924|     0.26737967|          0.5828877|      0.112299465|     real|
+--------------------+--------------------+---------+--------+---------------+-------------------+------------------+---------------+-------------------+-----------------+---------+
only showing top 1 row



# Logistic Regression

In [44]:
from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures=20000)
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx', 'length_in_words_scaled', 'average_word_length_scaled', 'capital_char_ratio_scaled', 'long_word_ratio_scaled', 'non_stop_word_ratio_scaled', 'punctuation_ratio_scaled'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
lr = LogisticRegression(regParam = 0.3)
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1, assembles, label_strIdx2, lr ,label_idxStr])


In [46]:
model = pipeline.fit(combined_train)

In [48]:
pred_old_lr = model.transform(old_test_split)
pred_new_lr = model.transform(new_test_split)
pred_combined_lr = model.transform(combined_test)

pred_old_lr.cache()
pred_new_lr.cache()
pred_combined_lr.cache()

DataFrame[Review: string, stemmed: array<string>, sentiment: float, polarity: string, length_in_words: int, average_word_length: float, capital_char_ratio: float, long_word_ratio: float, non_stop_word_ratio: float, punctuation_ratio: float, real_fake: string, length_in_words_vec: vector, average_word_length_vec: vector, capital_char_ratio_vec: vector, long_word_ratio_vec: vector, non_stop_word_ratio_vec: vector, punctuation_ratio_vec: vector, length_in_words_scaled: vector, average_word_length_scaled: vector, capital_char_ratio_scaled: vector, long_word_ratio_scaled: vector, non_stop_word_ratio_scaled: vector, punctuation_ratio_scaled: vector, rawFeatures: vector, TF_IDF: vector, polarity_idx: double, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double, article_class: string]

In [49]:
pred_combined_lr.select("review","polarity", "label", "prediction","article_class").show()

+--------------------+--------+-----+----------+-------------+
|              review|polarity|label|prediction|article_class|
+--------------------+--------+-----+----------+-------------+
| My wife and I's ...|positive|  1.0|       1.0|         fake|
|A few nights ago,...|positive|  1.0|       1.0|         fake|
|A few weeks ago w...|negative|  1.0|       1.0|         fake|
|A friend and I st...|negative|  1.0|       1.0|         fake|
|A hotel made for ...|positive|  1.0|       1.0|         fake|
|After arriving at...|negative|  1.0|       0.0|         fake|
|After considering...|negative|  1.0|       0.0|         fake|
|After reading all...|negative|  0.0|       0.0|         real|
|After searching f...|positive|  1.0|       1.0|         fake|
|After staying at ...|negative|  1.0|       1.0|         fake|
|Although the Inte...|negative|  1.0|       1.0|         fake|
|Amalfi Hotel Chic...|positive|  1.0|       0.0|         fake|
|Ambassador East i...|positive|  0.0|       1.0|       

In [50]:
pred_combined_lr.show()

+--------------------+--------------------+---------+--------+---------------+-------------------+------------------+---------------+-------------------+-----------------+---------+-------------------+-----------------------+----------------------+--------------------+-----------------------+---------------------+----------------------+--------------------------+-------------------------+----------------------+--------------------------+------------------------+--------------------+--------------------+------------+--------------------+-----+--------------------+--------------------+----------+-------------+
|              Review|             stemmed|sentiment|polarity|length_in_words|average_word_length|capital_char_ratio|long_word_ratio|non_stop_word_ratio|punctuation_ratio|real_fake|length_in_words_vec|average_word_length_vec|capital_char_ratio_vec| long_word_ratio_vec|non_stop_word_ratio_vec|punctuation_ratio_vec|length_in_words_scaled|average_word_length_scaled|capital_char_ratio_

In [51]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_our_data = eval.evaluate(pred_new_lr)
print("our data: ", acc_our_data)
acc_original_data = eval.evaluate(pred_old_lr)
print("original data: ", acc_original_data)
acc_original_data = eval.evaluate(pred_combined_lr)
print("combined data: ", acc_original_data)


our data:  0.770949720670391
original data:  0.8245033112582781
combined data:  0.8045738045738046


In [52]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10000,20000,50000]) \
    .addGrid(lr.regParam, [0.1, 0.3 ,0.5]) \
    .build()
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid,evaluator= MulticlassClassificationEvaluator(),numFolds=3,parallelism = 100 )  # use 3+ folds in practice

### ON REFINEMENT DO 5 FOLD? MAYBE JUST FOR BEST PRELIMINARY MODEL?###

In [53]:
cvModel_lr = crossval.fit(combined_train)

In [55]:
pred_old_lr = cvModel_lr.transform(old_test_split)
pred_new_lr = cvModel_lr.transform(new_test_split)
pred_combined_lr = cvModel_lr.transform(combined_test)

pred_old_lr.cache()
pred_new_lr.cache()
pred_combined_lr.cache()

DataFrame[Review: string, stemmed: array<string>, sentiment: float, polarity: string, length_in_words: int, average_word_length: float, capital_char_ratio: float, long_word_ratio: float, non_stop_word_ratio: float, punctuation_ratio: float, real_fake: string, length_in_words_vec: vector, average_word_length_vec: vector, capital_char_ratio_vec: vector, long_word_ratio_vec: vector, non_stop_word_ratio_vec: vector, punctuation_ratio_vec: vector, length_in_words_scaled: vector, average_word_length_scaled: vector, capital_char_ratio_scaled: vector, long_word_ratio_scaled: vector, non_stop_word_ratio_scaled: vector, punctuation_ratio_scaled: vector, rawFeatures: vector, TF_IDF: vector, polarity_idx: double, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double, article_class: string]

In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_our_data = eval.evaluate(pred_new_lr)
print("our data: ", acc_our_data)
acc_original_data = eval.evaluate(pred_old_lr)
print("original data: ", acc_original_data)
acc_original_data = eval.evaluate(pred_combined_lr)
print("combined data: ", acc_original_data)


our data:  0.7597765363128491
original data:  0.8278145695364238
combined data:  0.8024948024948025


In [86]:
params = [{
      p.name: v
      for p,
      v in m.items()
   }
   for m in cvModel_lr.getEstimatorParamMaps()
]
import pandas as pd

pd.DataFrame.from_dict([{
      cvModel_lr.getEvaluator().getMetricName(): metric,
      ** ps
   }
   for ps, metric in zip(params, cvModel_lr.avgMetrics)
])

,f1,numFeatures,regParam
5,0.777867,20000,0.5
8,0.780561,50000,0.5
2,0.781884,10000,0.5
3,0.782512,20000,0.1
0,0.785833,10000,0.1
4,0.785967,20000,0.3
6,0.786689,50000,0.1
1,0.788083,10000,0.3
7,0.789984,50000,0.3


In [76]:
print("Best regularization parameter: ", cvModel_lr.bestModel.stages[6]._java_obj.getRegParam())
print("Best hashing number of features: ", cvModel_lr.bestModel.stages[1]._java_obj.getNumFeatures())

In [77]:
from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures=best_numFeatures)
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx', 'length_in_words_scaled', 'average_word_length_scaled', 'capital_char_ratio_scaled', 'long_word_ratio_scaled', 'non_stop_word_ratio_scaled', 'punctuation_ratio_scaled'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
lr = LogisticRegression(regParam = best_regParam)
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1, assembles, label_strIdx2, lr ,label_idxStr])

In [78]:
best_model_lr = pipeline.fit(combined_train)

In [83]:
best_pred_old_lr = best_model_lr.transform(old_test_split)
best_pred_new_lr = best_model_lr.transform(new_test_split)
best_pred_combined_lr = best_model_lr.transform(combined_test)

In [85]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_our_data = eval.evaluate(best_pred_new_lr)
print("our data: ", acc_our_data)
acc_original_data = eval.evaluate(best_pred_old_lr)
print("original data: ", acc_original_data)
acc_original_data = eval.evaluate(best_pred_combined_lr)
print("combined data: ", acc_original_data)

0.8278145695364238
0.8278145695364238

our data:  0.7597765363128491
original data:  0.8278145695364238
combined data:  0.8024948024948025


0.8278145695364238

# SVC

In [59]:
from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
svc = LinearSVC(featuresCol="features",labelCol="label")
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline_svc = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1 , assembles, label_strIdx2, svc ,label_idxStr])

In [60]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10000,20000,50000]) \
    .addGrid(svc.regParam, [0.0001, 0.001, 0.01, 0.1, 1]) \
    .build()
crossval_svc = CrossValidator(estimator=pipeline_svc, estimatorParamMaps=paramGrid,evaluator= MulticlassClassificationEvaluator(),numFolds=2,parallelism = 100 )  # use 3+ folds in practice

In [61]:
cvModel_svc = crossval_svc.fit(combined_train)

In [72]:
pred_old_svc = cvModel_svc.transform(old_test_split)
pred_new_svc = cvModel_svc.transform(new_test_split)
pred_combined_svc = cvModel_svc.transform(combined_test)

pred_old_svc.cache()
pred_new_svc.cache()
pred_combined_svc.cache()

DataFrame[Review: string, stemmed: array<string>, sentiment: float, polarity: string, length_in_words: int, average_word_length: float, capital_char_ratio: float, long_word_ratio: float, non_stop_word_ratio: float, punctuation_ratio: float, real_fake: string, length_in_words_vec: vector, average_word_length_vec: vector, capital_char_ratio_vec: vector, long_word_ratio_vec: vector, non_stop_word_ratio_vec: vector, punctuation_ratio_vec: vector, length_in_words_scaled: vector, average_word_length_scaled: vector, capital_char_ratio_scaled: vector, long_word_ratio_scaled: vector, non_stop_word_ratio_scaled: vector, punctuation_ratio_scaled: vector, rawFeatures: vector, TF_IDF: vector, polarity_idx: double, features: vector, label: double, rawPrediction: vector, prediction: double, article_class: string]

In [73]:
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_new_data = eval.evaluate(pred_new_svc)
print("new data: ", acc_new_data)
acc_old_data = eval.evaluate(pred_old_svc)
print("old data: ", acc_old_data)
acc_combined_data = eval.evaluate(pred_combined_svc)
print("combined data: ", acc_combined_data)

new data:  0.7653631284916201
old data:  0.7947019867549668
combined data:  0.7837837837837838


In [74]:
params = [{
      p.name: v
      for p,
      v in m.items()
   }
   for m in cvModel_svc.getEstimatorParamMaps()
]
import pandas as pd

pd.DataFrame.from_dict([{
      cvModel_svc.getEvaluator().getMetricName(): metric,
      ** ps
   }
   for ps, metric in zip(params, cvModel_svc.avgMetrics)
])

,f1,numFeatures,regParam
0,0.728769,10000,0.0001
1,0.723906,10000,0.0010
2,0.740623,10000,0.0100
3,0.754587,10000,0.1000
4,0.755841,10000,1.0000
5,0.725729,20000,0.0001
6,0.725903,20000,0.0010
7,0.741347,20000,0.0100
8,0.745352,20000,0.1000
9,0.758531,20000,1.0000


In [122]:
print("Best regularization parameter: ", cvModel_svc.bestModel.stages[6]._java_obj.getRegParam())
print("Best hashing number of features: ", cvModel_svc.bestModel.stages[1]._java_obj.getNumFeatures())

ConnectionRefusedError: ignored

# Random Forest

In [123]:
#greg

from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
rfc = RandomForestClassifier(featuresCol="features",labelCol="label")
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline_rfc = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1 , assembles, label_strIdx2, rfc ,label_idxStr])

ConnectionRefusedError: ignored

In [88]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10000,20000,50000]) \
    .addGrid(rfc.maxDepth, [3, 5, 9]) \
    .addGrid(rfc.numTrees, [10, 20, 50]) \
    .build()
crossval_rfc = CrossValidator(estimator=pipeline_rfc, estimatorParamMaps=paramGrid,evaluator= MulticlassClassificationEvaluator(),numFolds=2,parallelism = 100 )  # use 3+ folds in practice

In [89]:
cvModel_rfc = crossval_rfc.fit(combined_train)

Py4JJavaError: ignored

In [ ]:
pred_old_rfc = cvModel_rfc.transform(old_test_split)
pred_new_rfc = cvModel_rfc.transform(new_test_split)
pred_combined_rfc = cvModel_rfc.transform(combined_test)

pred_old_rfc.cache()
pred_new_rfc.cache()
pred_combined_rfc.cache()

In [ ]:
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_new_data = eval.evaluate(pred_new_rfc)
print("new data: ", acc_new_data)
acc_old_data = eval.evaluate(pred_old_rfc)
print("old data: ", acc_old_data)
acc_combined_data = eval.evaluate(pred_combined_rfc)
print("combined data: ", acc_combined_data)

In [ ]:
params = [{
      p.name: v
      for p,
      v in m.items()
   }
   for m in cvModel_rfc.getEstimatorParamMaps()
]
import pandas as pd

pd.DataFrame.from_dict([{
      cvModel_rfc.getEvaluator().getMetricName(): metric,
      ** ps
   } for ps, metric in zip(params, cvModel_rfc.avgMetrics)
])

In [ ]:
print("Best regularization parameter: ", cvModel_rfc.bestModel.stages[6]._java_obj.getRegParam())
print("Best hashing number of features: ", cvModel_rfc.bestModel.stages[1]._java_obj.getNumFeatures())

# Gradient Boosting

In [ ]:
#greg

from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
gbtc = GBTClassifier(featuresCol="features",labelCol="label")
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline_gbtc = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1 , assembles, label_strIdx2, gbtc ,label_idxStr])

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10000,16384,32768]) \
    .addGrid(gbtc.maxDepth, [3, 5, 7]) \
    .addGrid(gbtc.stepSize, [0.1, 0.5, 1]) \
    .build()
crossval_gbtc = CrossValidator(estimator=pipeline_gbtc, estimatorParamMaps=paramGrid,evaluator= MulticlassClassificationEvaluator(),numFolds=2,parallelism = 100 )  # use 3+ folds in practice

In [ ]:
cvModel_gbtc = crossval_gbtc.fit(train_split)

In [ ]:
pred_gbtc = cvModel_gbtc.transform(test_split)
pred_original_gbtc = cvModel_gbtc.transform(df_test)
pred_gbtc.cache()
pred_original_gbtc.cache()

In [ ]:
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_our_data_gbtc = eval.evaluate(pred_gbtc)
print("our data: ", acc_our_data_gbtc)
acc_original_data_gbtc = eval.evaluate(pred_original_gbtc)
print("original data: ", acc_original_data_gbtc)

In [ ]:
params = [{
      p.name: v
      for p,
      v in m.items()
   }
   for m in cvModel_gbtc.getEstimatorParamMaps()
]
import pandas as pd

pd.DataFrame.from_dict([{
      cvModel_gbtc.getEvaluator().getMetricName(): metric,
      ** ps
   }
   for ps, metric in zip(params, cvModel_gbtc.avgMetrics)
])

In [ ]:
#greg

from pyspark.ml.feature import HashingTF, IDF, IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

columns_to_be_scaled = ['length_in_words', 'average_word_length', 'capital_char_ratio', 'long_word_ratio', 'non_stop_word_ratio', 'punctuation_ratio']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_be_scaled]
scalers  = [StandardScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_be_scaled]
scaling_pipeline = Pipeline(stages=assemblers + scalers)

hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="TF_IDF", minDocFreq=2)
label_strIdx1 = StringIndexer(inputCol="polarity", outputCol="polarity_idx")
assembles = VectorAssembler(inputCols = ['TF_IDF','sentiment','polarity_idx'],outputCol="features")
label_strIdx2 = StringIndexer(inputCol="real_fake", outputCol="label")
nb = NaiveBayes(featuresCol="features",labelCol="label", modelType="gaussian")
label_idxStr = IndexToString(inputCol = "label", outputCol = "article_class")

pipeline_nb = Pipeline(stages=[scaling_pipeline, hashingTF, idf,label_strIdx1 , assembles, label_strIdx2, nb ,label_idxStr])

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10000,16384,32768]) \
    .addGrid(nb.smoothing, [0.1, 1, 10]) \
    .build()
crossval_nb = CrossValidator(estimator=pipeline_nb, estimatorParamMaps=paramGrid,evaluator= MulticlassClassificationEvaluator(),numFolds=2,parallelism = 100 )  # use 3+ folds in practice

In [ ]:
cvModel_nb = crossval_nb.fit(train_split)

In [ ]:
pred_nb = cvModel_nb.transform(test_split)
pred_original_nb = cvModel_nb.transform(df_test)
pred_nb.cache()
pred_original_nb.cache()

In [ ]:
eval = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
acc_our_data_nb = eval.evaluate(pred_gbtc)
print("our data: ", acc_our_data_nb)
acc_original_data_nb = eval.evaluate(pred_original_nb)
print("original data: ", acc_original_data_nb)

In [ ]:
params = [{
      p.name: v
      for p,
      v in m.items()
   }
   for m in cvModel_nb.getEstimatorParamMaps()
]
import pandas as pd

pd.DataFrame.from_dict([{
      cvModel_nb.getEvaluator().getMetricName(): metric,
      ** ps
   }
   for ps, metric in zip(params, cvModel_nb.avgMetrics)
])